In [1]:
using ExprOptimization
using NPZ
using Plots, Parameters, BenchmarkTools
using ExprOptimization, Random
using LinearAlgebra
using Distributed

## Linkage Kinematics
Solving the 2D kinematics of linkage problems can be done using basic trignometry under the assumption that all linkages are completely rigid in all directions. This is ofcourse contingent upon the mechanism in question being valid, that is the mechanims must have one degree of freedom (or the same degree of freedom as the number of independant actuators) and not be locking for the given motor position. To solve the kinematics of mechaisms we can first look at the simplified unit of three nodes in the system.

<img src="https://i.ibb.co/9nmPkT5/linkage-breakdown.png" alt="linkage breakdown" border="0">

Suppose we know the positions of nodes 1 and 2 at the current time t. Given we have defined the mechanim we also know the lengths of the linkages connecting 1 to 3 and 2 to 3. In the way we define linkage mechinasms instead of storing linkage lengths (which does not define a full a unique mechanims) we store the positions of all nodes at time t=0s or the initial positions of the linkages, this means that we store the structure (Adjacency matrix) and initial positions for a given mechanim. From here to get to the linkage lengths we can simply compute the distance between the intial positions of any two connected nodes. Moving forward the following nomaenclature is used:

$X_{0,i} = $ Initial Position of Node $i$<br>
$X_{i}(t) = $ Position of Node $i$ at time $t$<br>
$C_{i,j} = $ The $i,j$ element of the Adjacency matrix. 1 inf $i$ and $j$ are connected and 0 otherwise.<br> 
$G_{i,j} = \frac{||X_{0,i}-X_{0,j}||_2}{C_{i,j}}$ Length of linkage connecting Nodes $i$ and $j$ ($\infty$ if the node are not connected)<br>
$R(\phi) = $ 2D (rotation about $\underline{\hat{e_z}}$) rotation matrix of angle phi

Now consired the system of three nodes we discussed prior. Here we assume that $X_{1}$ and $X_{2}$ are known at the current time t and we want to find the $X_{3}$ at the given time. To do this we can use the cosine rule to find $\theta$:

<img src="https://i.ibb.co/7SmbtWT/linkage-breakdown-cosine.png" alt="linkage-breakdown-cosine" border="0">

We can write:

$cos(\theta) = \frac{G_{1,3}^2+||X_{1}-X_{2}||^2_2-G_{2,3}^2}{2 G_{1,3}^2 ||X_{1}-X_{2}||^2_2} = \frac{||X_{0,1}-X_{0,3}||^2_2 + ||X_{1}-X_{2}||^2_2 - ||X_{0,2}-X_{0,3}||^2_2}{2 ||X_{0,1}-X_{0,3}||_2 ||X_{1}-X_{2}||_2}$

which means that:

$|\theta| = cos^{-1}(\frac{||X_{0,1}-X_{0,3}||^2_2 + ||X_{1}-X_{2}||^2_2 - ||X_{0,2}-X_{0,3}||^2_2}{2 ||X_{0,1}-X_{0,3}||_2 ||X_{1}-X_{2}||_2})$

However, this only gets us the absolute value of theta as $cos^{-1}$ has a range of $[0,2\pi]$. so we must also determine which way the angle should be to solve this system.

<img src="https://i.ibb.co/x5zDC53/linkage-breakdown-cosine-2.png" alt="linkage-breakdown-cosine-2" border="0">

To find the sign of the angle we shall look at the initial positions to see if 3 started above the 1 and 2 or below them. This can be easily determined by:

$sign(\theta) = sign([(X_{0,1}-X_{0,3})\times(X_{0,1}-X_{0,2})].\underline{\hat{e_z}})$

So we have the final equation for $\theta$:

$\theta = sign([(X_{0,1}-X_{0,3})\times(X_{0,1}-X_{0,2})].\underline{\hat{e_z}}) \times cos^{-1}(\frac{||X_{0,1}-X_{0,3}||^2_2 + ||X_{1}-X_{2}||^2_2 - ||X_{0,2}-X_{0,3}||^2_2}{2 ||X_{0,1}-X_{0,3}||_2 ||X_{1}-X_{2}||_2})$

The final step in finding the position of node 3 is to rotate the unit vector in the direction connecting 1 to 2 by $\theta$ and move in this rotated direction by the amount eaqual to $G_{1,3}$ from $X_{1}$ to obtain the position of node 3 at time t:

$X_{3} = X_{1} + R(\theta) \frac{(X_{2}-X_{1})||X_{0,3}-X_{0,1}||_2}{||X_{2}-X_{1}||_2}$

or 

$X_{3} = X_{1} + R(sign([(X_{0,1}-X_{0,3})\times(X_{0,1}-X_{0,2})].\underline{\hat{e_z}}) \times cos^{-1}(\frac{||X_{0,1}-X_{0,3}||^2_2 + ||X_{1}-X_{2}||^2_2 - ||X_{0,2}-X_{0,3}||^2_2}{2 ||X_{0,1}-X_{0,3}||_2 ||X_{1}-X_{2}||_2})) \frac{(X_{2}-X_{1})||X_{0,3}-X_{0,1}||_2}{||X_{2}-X_{1}||_2}$

Given this overall equation to solve the kinematics of a mechanims at time $t$ we simply start from the nodes with known positions, which are ground nodes (Always stationary at their initial positions) and the motor node (with known position given the motor/actuator curve) and see which unkown nodes have connections to two known nodes. Using the above equation we can solve for this set of nodes and add them to our list of known nodes and repeat the process with the expanded list of knowns. This means that nodes away from ground nodes and motor have a long chain to the position of the motor and their overall equation grows depending on the number of steps it takes to solve for them. A major issue with this function is that when the mechanism locks the $cos(\theta)$ value goes above 1 meaning the solution does not exist which means that this function returns an error. This makes it difficult to come up with a loss for symbolic regression when this occurs.

Now using this knowledge a dataset of 100 mechanims and their solutions for 200 equal timesteps for a full rotation of the motor are given below:

In [2]:
x1s = npzread("x1s.npy")
x2s = npzread("x2s.npy")
x3s = npzread("x3s.npy")
x0_1s = npzread("x0_1s.npy")
x0_2s = npzread("x0_2s.npy")
x0_3s = npzread("x0_3s.npy")

1908200×2 Array{Float64,2}:
 0.306585  0.725864
 0.306585  0.725864
 0.306585  0.725864
 0.306585  0.725864
 0.306585  0.725864
 0.306585  0.725864
 0.306585  0.725864
 0.306585  0.725864
 0.306585  0.725864
 0.306585  0.725864
 0.306585  0.725864
 0.306585  0.725864
 0.306585  0.725864
 ⋮         
 0.576007  0.648505
 0.576007  0.648505
 0.576007  0.648505
 0.576007  0.648505
 0.576007  0.648505
 0.576007  0.648505
 0.576007  0.648505
 0.576007  0.648505
 0.576007  0.648505
 0.576007  0.648505
 0.576007  0.648505
 0.576007  0.648505

To use symbolic regression we first define a few functions that are needed:

In [3]:
function norm(x)
    sqrt(x*x')[1]
end

function cross2D(x,y)
    x[1,1]*y[1,2] - x[1,2]*y[1,1]
end

cross2D (generic function with 1 method)

In [4]:
function R(phi)
    [cos(phi) -sin(phi);sin(phi) cos(phi)]
end
function Rotate(x,phi)
    x*R(phi)
end

Rotate (generic function with 1 method)

The functions below are the exact solutions:

In [5]:
function get_pos(x_1::Any, x_2::Any, x0_1::Any, x0_2::Any, x0_3::Any)
    (((((x_2)-(x_1))/(norm((x_2)-(x_1))))*(norm((x0_1)-(x0_3))))*(R((sign(cross2D(x0_1-x0_3,x0_1-x0_2)))*(acos(((((norm((x_2)-(x_1)))^(2))+((norm((x0_1)-(x0_3)))^(2)))-((norm((x0_2)-(x0_3)))^(2)))/(((2)*(norm((x_2)-(x_1))))*(norm((x0_1)-(x0_3)))))))))+(x_1)
end

function get_cos_theta(x_1::Any, x_2::Any, x0_1::Any, x0_2::Any, x0_3::Any)
    ((((norm((x_2)-(x_1)))^(2))+((norm((x0_1)-(x0_3)))^(2)))-((norm((x0_2)-(x0_3)))^(2)))/(((2)*(norm((x_2)-(x_1))))*(norm((x0_1)-(x0_3))))
end

get_cos_theta (generic function with 1 method)

Now we can define losses for the problem of finding the cosine of the angle and the problem of the overall equation:

Now define a grammer:

In [6]:
const grammar = @grammar begin
    Real = norm(x_1-x_2) | norm(x0_1-x0_3) | norm(x0_3-x0_2)
    Real = Real ^ 2
    Real = Real + Real
    Real = Real * Real
    Real = Real / Real
    Real = Real - Real
    Real = 1 | 2
end

1: Real = norm(x_1 - x_2)
2: Real = norm(x0_1 - x0_3)
3: Real = norm(x0_3 - x0_2)
4: Real = Real ^ 2
5: Real = Real + Real
6: Real = Real * Real
7: Real = Real / Real
8: Real = Real - Real
9: Real = 1
10: Real = 2


In [7]:
const S = SymbolTable(grammar)

Dict{Symbol,Any} with 7 entries:
  :+    => +
  :^    => ^
  :/    => /
  :Real => Real
  :norm => norm
  :-    => -
  :*    => *

Compute $cos(\theta)$ values:

In [8]:
cosphis = []

Any[]

In [9]:
for i = 1:1908200
    x_1 = x1s[i,:]'
    x_2 = x2s[i,:]'
    x_3 = x3s[i,:]'
    x0_1 = x0_1s[i,:]'
    x0_2 = x0_2s[i,:]'
    x0_3 = x0_3s[i,:]'
    
    push!(cosphis,get_cos_theta(x_1, x_2, x0_1, x0_2, x0_3))
end

In [10]:
function loss(tree::RuleNode, grammar::Grammar)
    ex = get_executable(tree, grammar)
    los = 0.0
    for i = 1:2000
        x_1 = x1s[i,:]'
        x_2 = x2s[i,:]'
        x_3 = x3s[i,:]'
        x0_1 = x0_1s[i,:]'
        x0_2 = x0_2s[i,:]'
        x0_3 = x0_3s[i,:]'
        S[:x_1] = x_1
        S[:x_2] = x_2
        S[:x0_1] = x0_1
        S[:x0_2] = x0_2
        S[:x0_3] = x0_3
        cosphi = cosphis[i]
        predicted_phi = Core.eval(S,ex)
        los += abs2(cosphi-predicted_phi)
    end
    los
end

loss (generic function with 1 method)

Now we can first apply GeneticProgramming to the simplified problem of finding the angle:

In [11]:
?GeneticProgram

search: GeneticProgram GeneticPrograms



```
GeneticProgram
```

Genetic Programming.

# Arguments

  * `pop_size::Int`: population size
  * `iterations::Int`: number of iterations
  * `max_depth::Int`: maximum depth of derivation tree
  * `p_reproduction::Float64`: probability of reproduction operator
  * `p_crossover::Float64`: probability of crossover operator
  * `p_mutation::Float64`: probability of mutation operator
  * `init_method::InitializationMethod`: initialization method
  * `select_method::SelectionMethod`: selection method
  * `track_method::TrackingMethod`: additional tracking, e.g., track top k exprs (default: no additional tracking)


In [12]:
Random.seed!()
p = GeneticProgram(500,100,8,0.3,0.3,0.4)
results_ce = optimize(p, grammar, :Real, loss)
(results_ce.expr, results_ce.loss)

(:((((norm(x0_1 - x0_3) + norm(x_1 - x_2)) - norm(x0_3 - x0_2) ^ 2) - ((norm(x0_3 - x0_2) / (norm(x0_1 - x0_3) + norm(x_1 - x_2))) ^ 2) ^ 2) - norm(x0_1 - x0_3) / (norm(x0_1 - x0_3) + (2 + (norm(x0_1 - x0_3) / norm(x_1 - x_2)) ^ 2))), 26.35254719131084)

As we can see this problem is possible to solve using symbolic regression but the overall problem of solving the full equation is rather difficult:

In [13]:
const grammar2 = @grammar begin
    vec = x_1 | x_2 | x0_1 | x0_2 | x0_3
    vec = Real * vec
    vec = vec / Real
    vec = vec + vec
    vec = vec - vec
    vec = Rotate(vec,Real)
    Real = norm(vec)
    Real = cross2D(vec,vec)
    Real = Real ^ Real
    Real = Real + Real
    Real = Real * Real
    Real = Real / Real
    Real = Real - Real
    Real = acos(Real)
    Real = sign(Real)
#     Real = asin(Real)
#     Real = atan(Real)
#     Real = sin(Real)
#     Real = cos(Real)
#     Real = tan(Real)
    Real = 1|2
end

1: vec = x_1
2: vec = x_2
3: vec = x0_1
4: vec = x0_2
5: vec = x0_3
6: vec = Real * vec
7: vec = vec / Real
8: vec = vec + vec
9: vec = vec - vec
10: vec = Rotate(vec, Real)
11: Real = norm(vec)
12: Real = cross2D(vec, vec)
13: Real = Real ^ Real
14: Real = Real + Real
15: Real = Real * Real
16: Real = Real / Real
17: Real = Real - Real
18: Real = acos(Real)
19: Real = sign(Real)
20: Real = 1
21: Real = 2


In [14]:
const S = SymbolTable(grammar2)

Dict{Symbol,Any} with 12 entries:
  :+       => +
  :/       => /
  :^       => ^
  :*       => *
  :vec     => vec
  :Rotate  => Rotate
  :cross2D => cross2D
  :acos    => acos
  :sign    => sign
  :-       => -
  :norm    => norm
  :Real    => Real

In [15]:
function loss(tree::RuleNode, grammar::Grammar)
    ex = get_executable(tree, grammar)
    los = 0.0
    for i = 1:20000
        x_1 = x1s[i,:]'
        x_2 = x2s[i,:]'
        x_3 = x3s[i,:]'
        x0_1 = x0_1s[i,:]'
        x0_2 = x0_2s[i,:]'
        x0_3 = x0_3s[i,:]'
        S[:x_1] = x_1
        S[:x_2] = x_2
        S[:x0_1] = x0_1
        S[:x0_2] = x0_2
        S[:x0_3] = x0_3
        try
            x_3_p = Core.eval(S,ex)
            los += norm(x_3_p - x_3)
        catch
            los += 1e10
        end
    end
    los
end

loss (generic function with 1 method)

In [35]:
Random.seed!()
p = CrossEntropy(100,50,10,50)
results_ce = optimize(p, grammar2, :vec, loss)
(results_ce.expr, results_ce.loss)

(:x0_3, 0.0696569861591034)